In [14]:
import sympy
x0,x,v,v0,t,x0 = sympy.symbols('x_0 x v v_0 t x_0')

### Quick note on mutivariate gaussians
    - Mutivariate gaussians express the correlation between multiple random variables, such as position and velocity of a vehicle.
    - Correlation between variables drastically improves the posterior.
        - if you roughly know position and velocity, BUT they are correlated, then the estimate can be very accurate.

## For Newtonian Systems

You can describe the system using Newton's equations of motion.
Specifically:
    $$x = vt + x_0$$
    
Incorporating acceleration we get:

$$x = \frac{1}{2}at^2 + v_0t + x_0$$

And Assuming jerk we get:

$$x = \frac{1}{6}jt^3 + \frac{1}{2}a_0t^2 + v_0t + x_0$$

These equations come from solving a differential equation

## Kalman Filter algorithm

#### Initialization
    1. initialize the state of the filter
    2. initialize our belief in the system
    
#### Predict
    1. use the process model to predict the state at next dt
    2. Adjust belief to account for uncertainty in prediction

#### Update
    1. get a measurement/ belief in its accuracy
    2. compute residual between state and measurement
    3. compute scaling factor to weigh prediction and measure
    4. set state based of this scaling factor
    5. update belief based on how certain we are in the measurement
    


## Kalman Filter Equations

Predict
    Multivariate
$$\bar x = Fx + Bu$$
$$\bar P = FPF^T + Q$$

x,P: are the state mean and covariance
F: State transition function. When multiplied by x it computes the prior
Q: process covariance
B,u: used to model the control system

Update

$$y = z - H\bar x$$
$$K = \bar P H^T(H\bar P H^T + R)^-1$$
$$x = \bar x + Ky$$
$$P = (I - KH)\bar P$$

H: is the measurement function

z,R: measurement mean and noise covariance

y,K: residual and kalman gain

## From System Dynamics Matrix (A) to State Transition Matrix (F)


The laplace transform in the form of:
$$ \Phi(t) = \mathcal{L}^{-1}[(s\mathbf{I} - \mathbf{A})^{-1}]$$



Can also use the matrix exponential method, which utilizes a taylor series approximation to compute the state transition matrix.This method is for time-invariant systems, however if the rate of change of the system is small, a small enough time step should still provide an accurate approximation. 

Or use a generalized ode solver, turn A into differential equations and use numerical techniques.



### Numerical Method for finding F and Q

van Loan's method

Given a continous model:
$$\dot x = Ax + Gw$$

assuming $w$ is unity white noise, you can use this method. The method is implemented in filterpy.common under van_loan_discretization

## Modeling Process noise Q 

### Continous white noise
We can assume continous unity white noise and discretize it to calculate Q, which defines how much noise is added to our system over a time step. Given F, the state transition matrix, we can project the continous noise model onto F based on the instant time t.

$$\mathbf Q = \int_0^{\Delta t} \mathbf F(t)\mathbf{Q_c}\mathbf F^\mathsf{T}(t) dt$$

and we define $$Q_c$$ as a matrix times the spectral density of the white noise.

In [19]:
from sympy import (init_printing, Matrix, MatMul, 
                   integrate, symbols)

init_printing(use_latex='mathjax')
dt, phi = symbols('\Delta{t} \Phi_s')
F_k = Matrix([[1, dt, dt**2/2],
              [0,  1,      dt],
              [0,  0,       1]])
Q_c = Matrix([[0, 0, 0],
              [0, 0, 0],
              [0, 0, 1]])*phi

Q = integrate(F_k * Q_c * F_k.T, (dt, 0, dt))

# factor phi out of the matrix to make it more readable
Q = Q / phi
MatMul(Q, phi)

⎡         5           4           3⎤      
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────  ──────────⎥      
⎢    20          8           6     ⎥      
⎢                                  ⎥      
⎢         4           3           2⎥      
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────  ──────────⎥⋅\Phiₛ
⎢    8           3           2     ⎥      
⎢                                  ⎥      
⎢         3           2            ⎥      
⎢\Delta{t}   \Delta{t}             ⎥      
⎢──────────  ──────────  \Delta{t} ⎥      
⎣    6           2                 ⎦      

The above method requires us to know the spectral density, but we can use a piece-wise noise model that selects noise in an uncorrelated fashion. Advantages is that you just need to derive the 'noise gain' matrix and can use variance to model the system in terms of motion and noise we expect

## Piece-wise method

If youre modeling position and velocity for example, you can propagate the noise based on your system model. so change in velocity and position is:

$$delta_v = w(t)*\delta t$$
and 
$$delta_p = w(t)\delta \frac{t^2}{2}$$

put this in a matrix gamma and project it by the variance.

Here's an example for a second order system :

In [20]:
var = symbols('sigma^2_v')
v = Matrix([[dt**2 / 2], [dt], [1]])

Q = v * var * v.T

# factor variance out of the matrix to make it more readable
Q = Q / var
MatMul(Q, var)

⎡         4           3           2⎤    
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥    
⎢──────────  ──────────  ──────────⎥    
⎢    4           2           2     ⎥    
⎢                                  ⎥    
⎢         3                        ⎥    
⎢\Delta{t}            2            ⎥    
⎢──────────  \Delta{t}   \Delta{t} ⎥⋅σ²ᵥ
⎢    2                             ⎥    
⎢                                  ⎥    
⎢         2                        ⎥    
⎢\Delta{t}                         ⎥    
⎢──────────  \Delta{t}       1     ⎥    
⎣    2                             ⎦    

## Runge-Kutta implementation

In [21]:
def runge_kutta4(y, x, dx, f):
    """computes 4th order Runge-Kutta for dy/dx.
    y is the initial value for y
    x is the initial value for x
    dx is the difference in x (e.g. the time step)
    f is a callable function (y, x) that you supply 
    to compute dy/dx for the specified values.
    """
    
    k1 = dx * f(y, x)
    k2 = dx * f(y + 0.5*k1, x + 0.5*dx)
    k3 = dx * f(y + 0.5*k2, x + 0.5*dx)
    k4 = dx * f(y + k3, x + dx)
    
    return y + (k1 + 2*k2 + 2*k3 + k4) / 6.